# Creole OvineHDSNPList
the `CREOLE_INIA_UY.zip` dataset is encode in *A/B* format and provide a `OvineHDSNPList.txt` file. Does this file keeps the same information of a manifest file? Is the information consistent for the SNPs in common with *50K* chip?

In [7]:
import re
import csv
import itertools
import collections

from src.features.smarterdb import Dataset, global_connection, VariantSheep
from src.features.utils import sanitize

conn = global_connection()

In [2]:
dataset = Dataset.objects(file="CREOLE_INIA_UY.zip").get()
snplist = dataset.working_dir / "OvineHDSNPList.txt"

try to detect if the alleles I have in file list are similar to ones I have in database. Print a few samples

In [23]:
with open(snplist) as handle:
    reader = csv.reader(handle, delimiter="\t")
    header = next(reader)
    header = [sanitize(column) for column in header]
    print(header)
    SnpLine = collections.namedtuple("SnpLine", header)
    counter = 0
    for record in reader:
        # fix elements
        record[header.index('snp')] = re.sub(
                r'[\[\]]',
                "",
                record[header.index('snp')])
        
        snpline = SnpLine._make(record)
        qs = VariantSheep.objects(name=snpline.name)
        if qs.count() > 0:
            variant = qs.get()
            location = variant.get_location(version="Oar_v3.1", imported_from="manifest")
            if snpline.snp != location.illumina or snpline.customer_strand != location.strand or snpline.ilmn_strand != location.illumina_strand:
            # if snpline.customer_strand != location.strand:
                counter += 1
                print(f"{snpline.name} {snpline.chr}:{snpline.position} {location.chrom}:{location.position} {snpline.snp} {location.illumina} {snpline.customer_strand} {location.strand} {snpline.ilmn_strand} {location.illumina_strand}")
                
        if counter > 20:
            break


['name', 'chr', 'position', 'index', 'snp', 'customer_strand', 'ilmn_strand']
250506CS3901300500001_1084.1 7:89431547 7:89431547 T/C A/G BOT BOT BOT TOP
CZ925803_293.1 6:100552392 6:100552392 T/C A/G BOT BOT BOT TOP
DU244782_175.1 3:59783119 3:59783119 T/C A/G BOT BOT BOT TOP
DU247466_220.1 13:9182853 13:9182853 T/A A/T BOT BOT BOT TOP
DU279054_461.1 25:255537 25:255537 T/G A/C BOT BOT BOT TOP
DU292684_282.1 20:10624434 20:10624434 T/C A/G BOT BOT BOT TOP
DU305004_417.1 4:70754178 4:70754178 T/C A/G BOT BOT BOT TOP
DU326574_557.1 2:12492887 2:12492887 T/C A/G BOT BOT BOT TOP
DU340136_162.1 18:53259877 18:53259877 T/C A/G BOT BOT BOT TOP
DU344050_171.1 1:96806235 1:96806235 T/C A/G BOT BOT BOT TOP
DU345394_399.1 18:60904691 18:60904691 T/G A/C BOT BOT BOT TOP
DU404011_458.1 18:54252674 18:54252674 T/C A/G BOT BOT BOT TOP
DU414375_444.1 15:23935021 15:23935021 T/C A/G TOP TOP BOT TOP
DU435573_466.1 13:30047921 13:30047921 T/C A/G BOT BOT BOT TOP
DU448891_645.1 10:10922307 10:10922307 T/G

The `customer_strand` and the `location.strand` are always equal (I suppose since they are the same *OARv3*). However `ilmn_strand` and `location.illumina_strand` can be different. In such case the allele is **reverse complemented**

## About coding convenction in general
Relying on what I see in database, if `illumina_strand` is in *BOT*, I need to reverse complement the allele to have an `illumina_top`. If `strand` is in *BOT*, the `allele` (not the *illumina*) seems to be the `illumina_forward` or `illumina` in reverse order (depending if it comes from *SNPchimp* or *manifest*)